# 네이버 부동산 단지정보크롤링2(단지정보)
매물번호,매물주소,매물순위,중개사목록,정보업체,중개업소전화번호,집주인매물갯수,중개업소주소

In [1]:
import requests
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from datetime import datetime , timedelta
import time
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile


In [4]:
#세션처리
user_agent = ""
session = requests.Session()
session.headers.update({'user-agent' : user_agent, 'referer' : None})

In [11]:
driver = webdriver.Chrome('./chromedriver')
driver.implicitly_wait(5)

In [12]:
driver.get("https://new.land.naver.com/")

In [7]:
단지리스트 = ['리버나인','래미안로이파크','한양']

In [14]:
#크롤링
실패단지 = []
단지목록 = []
매물목록 = []
순위목록 = []
업소목록 = []
씨피목록 = []
매물번호목록 = []
대표번호목록 = []
휴대전화목록 = []
집주인목록 =[]
주소목록 = []
예외목록 = ['부동산114', '피터팬의 좋은방구하기']

for i1 in range(len(단지리스트)):
    print("시작")

    tmp_단지목록 = []
    tmp_매물목록 = []
    tmp_씨피목록 = []
    tmp_예외목록= 0
    tmp_매물번호인덱스 = []
    tmp_매물이름 = []

    단지명 = 단지리스트[i1]
    driver.get("https://new.land.naver.com/")
    time.sleep(1)
    elem_search = driver.find_element_by_class_name("search_input")
    elem_search.clear()
    elem_search.send_keys(단지명)
    elem_search.send_keys(Keys.RETURN)

    if driver.find_element_by_tag_name("h3").text == "검색결과":
        print("검색결과 여러개 존재")
        단지검색 = driver.find_elements_by_class_name("title")
        for i2 in range(len(단지검색)):
            tmp_단지목록.append(단지검색[i2].text)
        print("단지목록 tmp저장 성공")
        if 단지리스트[i1] in tmp_단지목록:
            driver.find_elements_by_class_name("title")[tmp_단지목록.index(단지리스트[i1])].click()
            print("검색결과중 한개 선택 및 클릭")
            time.sleep(1)
            매물이름 = driver.find_elements_by_class_name("item_title")
            업소씨피내역 = driver.find_elements_by_class_name("agent_name")
            
            
        else:
            실패단지.append(단지리스트[i1])
            continue
        for i3 in range(len(매물이름)):
            단지목록.append(단지리스트[i1])
            print("단지목록 성공")
            매물목록.append(매물이름[i3].text)
            print("매물목록 성공")
            순위목록.append(str(i3 + 1) + "위")
            print("순위목록 성공")
            if i3 == 0:
                업소목록.append(업소씨피내역[(1)].text)
            else:
                업소목록.append(업소씨피내역[((i3 * 2) + 1)].text)
            print("업소목록 성공")
            씨피목록.append(업소씨피내역[i3 * 2].text[:-3])
            print("씨피목록1 성공")
            tmp_씨피목록.append(업소씨피내역[i3 * 2].text[:-3])
            
            try:
                
                if tmp_씨피목록[i3] in 예외목록 :
                    print('부동산114루투진입')
                    #time.sleep(1)
                    driver.find_elements_by_class_name("label.label--cp")[tmp_예외목록].click()
                    tmp_예외목록 += 1
                    정보내역 = driver.find_element_by_id("detailContents1")
                    표속성 = 정보내역.find_elements_by_class_name("table_th")
                    표값 = 정보내역.find_elements_by_class_name("table_td")
                    번호내역 = driver.find_element_by_class_name("text.text--number")
                    tmp_매물번호인덱스 =[]
                    try:
                        집주인내역 = driver.find_element_by_class_name("text.text--point")
                        for i4 in range(len(표속성)):
                            tmp_매물번호인덱스.append(표속성[i4].text)
                        매물번호인덱스 = tmp_매물번호인덱스.index('매물번호')
                        매물번호목록.append(표값[매물번호인덱스].text)
                        대표번호목록.append(번호내역.text[:-14])
                        휴대전화목록.append(번호내역.text[-13:])
                        집주인목록.append(집주인내역.text)
                        #time.sleep(0.5)
                        주소버튼 = driver.find_element_by_class_name("btn_agent_site").click()
                        주소내역 = driver.find_element_by_class_name("tooltip_site")
                        주소목록.append(주소내역.text)
                    except:
                        for i4 in range(len(표속성)):
                            tmp_매물번호인덱스.append(표속성[i4].text)
                        매물번호인덱스 = tmp_매물번호인덱스.index('매물번호')
                        매물번호목록.append(표값[매물번호인덱스].text)
                        대표번호목록.append(번호내역.text[:-14])
                        휴대전화목록.append(번호내역.text[-13:])
                        집주인목록.append("")
                        #time.sleep(0.5)
                        주소버튼 = driver.find_element_by_class_name("btn_agent_site").click()
                        주소내역 = driver.find_element_by_class_name("tooltip_site")
                        주소목록.append(주소내역.text)
                        

                else:
                    print('일반루트진입')
                    driver.find_elements_by_class_name("item_title")[i3].click()
                    정보내역 = driver.find_element_by_id("detailContents1")
                    표속성 = 정보내역.find_elements_by_class_name("table_th")
                    표값 = 정보내역.find_elements_by_class_name("table_td")
                    번호내역 = driver.find_element_by_class_name("text.text--number")
                    tmp_매물번호인덱스 =[]
                    try:
                        집주인내역 = driver.find_element_by_class_name("text.text--point")
                        for i4 in range(len(표속성)):
                            tmp_매물번호인덱스.append(표속성[i4].text)
                        매물번호인덱스 = tmp_매물번호인덱스.index('매물번호')
                        매물번호목록.append(표값[매물번호인덱스].text)
                        대표번호목록.append(번호내역.text[:-14])
                        휴대전화목록.append(번호내역.text[-13:])
                        집주인목록.append(집주인내역.text)
                        #time.sleep(0.5)
                        주소버튼 = driver.find_element_by_class_name("btn_agent_site").click()
                        주소내역 = driver.find_element_by_class_name("tooltip_site")
                        주소목록.append(주소내역.text)
                    except:
                        for i4 in range(len(표속성)):
                            tmp_매물번호인덱스.append(표속성[i4].text)
                        매물번호인덱스 = tmp_매물번호인덱스.index('매물번호')
                        매물번호목록.append(표값[매물번호인덱스].text)
                        대표번호목록.append(번호내역.text[:-14])
                        휴대전화목록.append(번호내역.text[-13:])
                        집주인목록.append("")
                        #time.sleep(0.5)
                        주소버튼 = driver.find_element_by_class_name("btn_agent_site").click()
                        주소내역 = driver.find_element_by_class_name("tooltip_site")
                        주소목록.append(주소내역.text)

            except:
                매물번호목록.append("")
                대표번호목록.append("")
                휴대전화목록.append("")
                집주인목록.append("")
                주소목록.append("")

    else:
        매물이름 = driver.find_elements_by_class_name("item_title")
        업소씨피내역 = driver.find_elements_by_class_name("agent_name")

        for i3 in range(len(매물이름)):
            단지목록.append(단지리스트[i1])
            매물목록.append(매물이름[i3].text)
            순위목록.append(str(i3 + 1) + "위")
            tmp_매물목록.append(매물이름[i3])
            if i3 == 0:
                업소목록.append(업소씨피내역[(1)].text)
            else:
                업소목록.append(업소씨피내역[((i3 * 2) + 1)].text)
            씨피목록.append(업소씨피내역[i3 * 2].text[:-3])
            tmp_씨피목록.append(업소씨피내역[i3 * 2].text[:-3])

            try:
                print("부동산114루트진입")
                if tmp_씨피목록[i3] in 예외목록:
                    time.sleep(1)
                    driver.find_elements_by_class_name("label.label--cp")[tmp_예외목록].click()
                    tmp_예외목록 += 1
                    정보내역 = driver.find_element_by_id("detailContents1")
                    표속성 = 정보내역.find_elements_by_class_name("table_th")
                    표값 = 정보내역.find_elements_by_class_name("table_td")
                    번호내역 = driver.find_element_by_class_name("text.text--number")
                    tmp_매물번호인덱스 =[]
                    try:
                        집주인내역 = driver.find_element_by_class_name("text.text--point")
                        for i4 in range(len(표속성)):
                            tmp_매물번호인덱스.append(표속성[i4].text)
                        매물번호인덱스 = tmp_매물번호인덱스.index('매물번호')
                        매물번호목록.append(표값[매물번호인덱스].text)
                        대표번호목록.append(번호내역.text[:-14])
                        휴대전화목록.append(번호내역.text[-13:])
                        집주인목록.append(집주인내역.text)
                        #time.sleep(0.5)
                        주소버튼 = driver.find_element_by_class_name("btn_agent_site").click()
                        주소내역 = driver.find_element_by_class_name("tooltip_site")
                        주소목록.append(주소내역.text)
                    except:
                        for i4 in range(len(표속성)):
                            tmp_매물번호인덱스.append(표속성[i4].text)
                        매물번호인덱스 = tmp_매물번호인덱스.index('매물번호')
                        매물번호목록.append(표값[매물번호인덱스].text)
                        대표번호목록.append(번호내역.text[:-14])
                        휴대전화목록.append(번호내역.text[-13:])
                        집주인목록.append("")
                        #time.sleep(0.5)
                        주소버튼 = driver.find_element_by_class_name("btn_agent_site").click()
                        주소내역 = driver.find_element_by_class_name("tooltip_site")
                        주소목록.append(주소내역.text)
                        

                else:
                    print("일반루트진입")
                    driver.find_elements_by_class_name("item_title")[i3].click()
                    정보내역 = driver.find_element_by_id("detailContents1")
                    표속성 = 정보내역.find_elements_by_class_name("table_th")
                    표값 = 정보내역.find_elements_by_class_name("table_td")
                    번호내역 = driver.find_element_by_class_name("text.text--number")
                    tmp_매물번호인덱스 =[]
                    try:
                        집주인내역 = driver.find_element_by_class_name("text.text--point")
                        for i4 in range(len(표속성)):
                            tmp_매물번호인덱스.append(표속성[i4].text)
                        매물번호인덱스 = tmp_매물번호인덱스.index('매물번호')
                        매물번호목록.append(표값[매물번호인덱스].text)
                        대표번호목록.append(번호내역.text[:-14])
                        휴대전화목록.append(번호내역.text[-13:])
                        집주인목록.append(집주인내역.text)
                        #time.sleep(0.5)
                        주소버튼 = driver.find_element_by_class_name("btn_agent_site").click()
                        주소내역 = driver.find_element_by_class_name("tooltip_site")
                        주소목록.append(주소내역.text)
                    except:
                        for i4 in range(len(표속성)):
                            tmp_매물번호인덱스.append(표속성[i4].text)
                        매물번호인덱스 = tmp_매물번호인덱스.index('매물번호')
                        매물번호목록.append(표값[매물번호인덱스].text)
                        대표번호목록.append(번호내역.text[:-14])
                        휴대전화목록.append(번호내역.text[-13:])
                        집주인목록.append("")
                        #time.sleep(0.5)
                        주소버튼 = driver.find_element_by_class_name("btn_agent_site").click()
                        주소내역 = driver.find_element_by_class_name("tooltip_site")
                        주소목록.append(주소내역.text)
                        
            except:
                매물번호목록.append("")
                대표번호목록.append("")
                휴대전화목록.append("")
                집주인목록.append("")
                주소목록.append("")
                
    print("{}종료 {}개 남음".format(단지리스트[i1],len(단지리스트)-(i1+1)))

시작
부동산114루트진입
일반루트진입
부동산114루트진입
일반루트진입
부동산114루트진입
일반루트진입
부동산114루트진입
일반루트진입
부동산114루트진입
일반루트진입
부동산114루트진입
일반루트진입
부동산114루트진입
일반루트진입
부동산114루트진입
일반루트진입
부동산114루트진입
일반루트진입
부동산114루트진입
일반루트진입
부동산114루트진입
일반루트진입
부동산114루트진입
일반루트진입
부동산114루트진입
일반루트진입
부동산114루트진입
일반루트진입
부동산114루트진입
일반루트진입
부동산114루트진입
일반루트진입
부동산114루트진입
일반루트진입
부동산114루트진입
일반루트진입
부동산114루트진입
일반루트진입
부동산114루트진입
일반루트진입
아크로리버뷰신반포종료 1개 남음
시작
부동산114루트진입
일반루트진입
부동산114루트진입
일반루트진입
부동산114루트진입
일반루트진입
부동산114루트진입
일반루트진입
부동산114루트진입
일반루트진입
부동산114루트진입
일반루트진입
부동산114루트진입
일반루트진입
부동산114루트진입
일반루트진입
부동산114루트진입
일반루트진입
부동산114루트진입
일반루트진입
부동산114루트진입
일반루트진입
부동산114루트진입
일반루트진입
부동산114루트진입
일반루트진입
부동산114루트진입
일반루트진입
부동산114루트진입
일반루트진입
부동산114루트진입
부동산114루트진입
일반루트진입
부동산114루트진입
일반루트진입
부동산114루트진입
일반루트진입
부동산114루트진입
래미안아름숲종료 0개 남음


In [15]:
df = pd.DataFrame()
df['단지목록'] = 단지목록
df['번호목록'] = 매물번호목록
df['매물목록'] = 매물목록
df['순위목록'] = 순위목록
df['업소목록'] = 업소목록
df['씨피목록'] = 씨피목록
df['대표목록'] = 대표번호목록
df['휴대목록'] = 휴대전화목록
df['집주인매물목록'] = 집주인목록
df['주소목록'] = 주소목록
df

,단지목록,번호목록,매물목록,순위목록,업소목록,씨피목록,대표목록,휴대목록,집주인매물목록,주소목록
0,아크로리버뷰신반포,2055219469,아크로리버뷰신반포 102동,1위,고터공인중개사사무소,부동산뱅크,02-537-4989,010-8811-7066,44건,서울시 서초구 잠원동 69-12 반포쇼핑타운6동 19호
1,아크로리버뷰신반포,2054997210,아크로리버뷰신반포 101동,2위,아크로소망공인중개사사무소,매경부동산,02-536-6700,010-5349-1186,14건,서울시 서초구 잠원동 65-4 1층 101호
2,아크로리버뷰신반포,2054996764,아크로리버뷰신반포 103동,3위,아크로소망공인중개사사무소,매경부동산,02-536-6700,010-5349-1186,14건,서울시 서초구 잠원동 65-4 1층 101호
3,아크로리버뷰신반포,2054478807,아크로리버뷰신반포 102동,4위,대성부동산,조인스랜드,,02-591-6677,6건,서울특별시 서초구 잠원동 나루터로 37 신사쇼핑105호
4,아크로리버뷰신반포,2054469409,아크로리버뷰신반포 102동,5위,반포자이행복한부동산공인중개사사무소,매경부동산,02-534-9800,010-8464-6782,9건,서울 서초구 반포동 20-51 반포자이주구중심상가2동 106호
5,아크로리버뷰신반포,2054433378,아크로리버뷰신반포 102동,6위,정공인중개사사무소,매경부동산,02-3477-9999,010-5422-0341,47건,서울 서초구 잠원동 60-6번지
6,아크로리버뷰신반포,2054417332,아크로리버뷰신반포 102동,7위,아크로소망공인중개사사무소,매경부동산,02-536-6700,010-5349-1186,14건,서울시 서초구 잠원동 65-4 1층 101호
7,아크로리버뷰신반포,2054473951,아크로리버뷰신반포 102동,8위,원베일리공인중개사사무소,매경부동산,02-593-2433,010-4516-2433,18건,서울시 서초구 잠원동 69-22 반포쇼핑타운 8동 1층24호
8,아크로리버뷰신반포,2054322842,아크로리버뷰신반포 102동,9위,명문공인중개사사무소,한경부동산,02-532-1007,010-7108-3651,6건,서울시 서초구 잠원동 69-19 반포쇼핑타운 7동 1층 9호
9,아크로리버뷰신반포,2053818739,아크로리버뷰신반포 101동,10위,아주공인중개사사무소,매경부동산,02-532-5111,010-9257-8982,18건,서울 서초구 잠원동 56-3 연합상가 103호


In [121]:
now = datetime.now()
writer = ExcelWriter('%s-%s-%s 로켓상위단지크롤링.xlsx'%(now.year, now.month, now.day))
df.to_excel(writer, '최종리스트')
writer.save()